# This is the simpliest pipeline of Fian.

## There are 3 phases

- Intent Detection

- Feature Selection

- Respond

# Phase 1 - Intent Detection

1. Extract user's intent (using TF-IDF + Logistic Regression)

First, for good measure, I will train the model first (train it once and give out a joblib file, but there for visualization)

Second, I will use the model to predict what's the user's intent


In [1]:
# Model Training: Intent Detection
import pandas as pd
import joblib
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

# STEP 1: Train from CSV and save model
def intent_LogReg(csv_path='intent_dataset.csv'):
    # Load CSV
    df = pd.read_csv(csv_path)

    # Make sure the expected columns exist
    if 'text' not in df.columns or 'intent' not in df.columns:
        raise ValueError("CSV must contain 'text' and 'intent' columns")

    texts = df['text'].astype(str).tolist()
    labels = df['intent'].astype(str).tolist()

    # Pipeline: TF-IDF + Logistic Regression
    model = Pipeline([
        ('tfidf', TfidfVectorizer()),
        ('clf', LogisticRegression())
    ])

    # Train and save
    model.fit(texts, labels)
    joblib.dump(model, 'intent_with_LogisticRegression.joblib')
    print("Model trained and saved as 'intent_with_LogisticRegression.joblib'.")

def extract_intent(text, threshold=0.6):
    try:
        model = joblib.load('intent_with_LogisticRegression.joblib')
        probs = model.predict_proba([text])[0]
        best_index = probs.argmax()
        confidence = probs[best_index]
        intent = model.classes_[best_index]

        if confidence >= threshold:
            return intent
        else:
            return "uncertain"

    except FileNotFoundError:
        return "error: model not found — please run intent_LogReg() first"

intent_LogReg()


Model trained and saved as 'intent_with_LogisticRegression.joblib'.


In [2]:
def chat_interface():
    print("=== Intent Detection Chat (type 'exit' to quit) ===")
    while True:
        user_input = input("You: ")
        if user_input.lower() in ['exit', 'quit']:
            print("Bot: Goodbye!")
            break

        intent = extract_intent(user_input)
        if intent == "uncertain":
            print("Bot: Hmm, not confident. Can you clarify that?")
        elif intent.startswith("error:"):
            print(f"Bot: {intent}")
            break
        else:
            print(f"Bot: I think your intent is: {intent}")


In [ ]:
chat_interface()

=== Intent Detection Chat (type 'exit' to quit) ===
